<a href="https://colab.research.google.com/github/Sanaz-salari/Multi-Layer-Perceptron-MLP/blob/main/Multi_Output_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scipy
!pip install scikit-optimize

     |████████████████████████████████| 100 kB 6.4 MB/s 


In [ ]:
## Importing required librariesimport sklearn
import skopt
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import RepeatedKFold

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense, Dropout

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
#############################################

# fix random seed for reproducibility
np.random.seed(5)
#Load data from local drive 
from google.colab import files
import pandas as pd
import io
uploaded = files.upload()

Saving Plasma_Dataset.csv to Plasma_Dataset.csv


In [ ]:
df = pd.read_csv(io.BytesIO(uploaded['Plasma_Dataset.csv']))
del df["run"] # drop "run" column
df.head()


,pressure,rf_power,electrode_gap,cci_flow,he_flow,o2_flow,etch_rate,etch_uniformity,oxide_selectivity,photoresist_selectivity
0,300,300,1.8,100,200,20,3491,14.2,6.48,2.01
1,200,400,1.8,100,50,10,3884,3.9,5.98,1.91
2,200,400,1.2,150,200,20,4931,24.8,5.39,1.85
3,300,400,1.8,150,200,20,4726,6.6,5.97,2.11
4,200,400,1.2,150,50,10,5089,12.4,5.61,2.16


In [ ]:
#Clean the data
df.isna().sum()

pressure                   0
rf_power                   0
electrode_gap              0
cci_flow                   0
he_flow                    0
o2_flow                    0
etch_rate                  0
etch_uniformity            0
oxide_selectivity          0
photoresist_selectivity    0
dtype: int64

In [ ]:
df2 = df.copy()
#Split the data into training and test sets
df2_x = df.sample(frac=0.8, random_state=0)
#df_x = test_dataset.copy()
df2_y = df2_x[['etch_rate','etch_uniformity','oxide_selectivity','photoresist_selectivity']]
df2_x.drop(['etch_rate','etch_uniformity','oxide_selectivity','photoresist_selectivity'], axis = 1, inplace = True)

df2_x_test = df.drop(df2_x.index)
df2_y_test = df2_x_test[['etch_rate','etch_uniformity','oxide_selectivity','photoresist_selectivity']]
df2_x_test.drop(['etch_rate','etch_uniformity','oxide_selectivity','photoresist_selectivity'], axis = 1, inplace = True)

print(df2_x.shape, df2_y.shape)
print(df2_x_test.shape, df2_y_test.shape)

(42, 6) (42, 4)
(11, 6) (11, 4)


In [ ]:
# Normalization:
df2_x_n=(df2_x - df2_x.min())/(df2_x.max()-df2_x.min())
df2_y_n=(df2_y - df2_y.min())/(df2_y.max()-df2_y.min())
X2 = df2_x_n.to_numpy()
y2 = df2_y_n.to_numpy()
Y2 = df2_y.to_numpy()

# Normalization on test data
df2_x_test_n=(df2_x_test - df2_x.min())/(df2_x.max()-df2_x.min())
df2_y_test_n=(df2_y_test - df2_y.min())/(df2_y.max()-df2_y.min())


X2_test = df2_x_test_n.to_numpy()
y2_test = df2_y_test_n.to_numpy()
Y2_test = df2_y_test.to_numpy()

In [ ]:
# Define & Compile Keras Model w regulizers for Hyper-Parameter Optimization
from tensorflow.keras import regularizers
# Defining various initialization parameters for MLP model
num_features = df2_x_n.shape[1]; num_classes = df2_y.shape[1] 

# Let's create a helper function first which builds the model with various parameters.
def get_model_reg(dense_0_neurons, dense_1_neurons, dropout_rate0, dropout_rate, input_dim, num_classes):  #dropout_rate0, 
    # Builds a Sequential MLP model using Keras and returns it
    
    # Define the keras model
    model = Sequential()
    model.add(Dense(dense_0_neurons, input_dim=input_dim, activation='relu', 
                    kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-4),
                    bias_regularizer=regularizers.l2(1e-4),
                    activity_regularizer=regularizers.l2(1e-4),
                    name="dense_1"))
    model.add(Dropout(dropout_rate0, name="dropout0"))
    model.add(Dense(dense_1_neurons, activation='relu',
                    kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-4),
                    bias_regularizer=regularizers.l2(1e-4),
                    activity_regularizer=regularizers.l2(1e-4),
                    name="dense_2"))
                    
    model.add(Dropout(dropout_rate, name="dropout"))
    model.add(Dense(num_classes, name="dense_3"))
    # Compile the keras model for a specified number of epochs.
    model.compile(loss='mean_squared_error',
                  optimizer='adam',
                  metrics=['mse'])
    return model

In [ ]:
# Setup Keras Model with layer Regularizers and cross validation for Scikit-Optimizer to implement on train data (X2, y2) { we use test data(x2_test, y2_test) to evaluate the model}
import skopt
from skopt import gp_minimize
# Specify `Static' Parameters
STATIC_PARAMS = {num_features, num_classes}

# Bounded region of parameter space
# The list of hyper-parameters we want to optimize. For each one we define the
# bounds, the corresponding scikit-learn parameter name, as well as how to
# sample values from that dimension (`'log-uniform'` for the dropout_rate)
SPACE = [skopt.space.Integer(2, 12, name='dense_0_neurons'),
         skopt.space.Integer(2, 12, name='dense_1_neurons'),
         skopt.space.Real(0.0, 0.4, name='dropout_rate'),
         skopt.space.Real(0.0, 0.4, name='dropout_rate0')]

# This decorator allows your objective function to receive a the parameters as
# keyword arguments. This is particularly convenient when you want to set
# scikit-learn estimator parameters         
@skopt.utils.use_named_args(SPACE)
         
# Define objective for optimization
def objective3(**params):
  
    # All parameters: 
    #all_params = {**params, **STATIC_PARAMS}
    results = list()
    # define evaluation procedure
    cv = RepeatedKFold(n_splits=5, n_repeats=4, random_state=1)
    # enumerate folds
    for train_ix, test_ix in cv.split(X2):
      # prepare data
      X_train, X_test = X2[train_ix], X2[test_ix]
      y_train, y_test = y2[train_ix], y2[test_ix]
		# define model
      model = get_model_reg(params["dense_0_neurons"], params["dense_1_neurons"], params["dropout_rate0"], params["dropout_rate"],num_features, num_classes)  #params["dropout_rate0"],      

      # Fit keras model
      history = model.fit(X_train, y_train, epochs=32, verbose=0)
      loss_epoch = history.history['loss']
    
    #sklearn.metrics.r2_score(y_true, y_pred)
    # Evaluate the model with the eval dataset.
      score = model.evaluate(X_test, y_test, verbose=0 ) ###, batch_size=4, verbose=0) # thei evaluation is a preliminary evaluation because it is done on the same data used for building the model
      
      # store result
      #print('>%.3f' % score[0])
      results.append(score[0])
      
    #test_loss = model.evaluate(X_test, y_test)
      print('CV MSE: %.3f:' % score[0])
    print('Cross validation average MSE is %.3f:' % np.mean(results))
    # Return the accuracy.
    return np.mean(results)

###########################################################################
###########################################################################
#Run Scikit-Optimizer
import warnings
warnings.filterwarnings('ignore')

results_gp = skopt.gp_minimize(objective3,     # the function to minimize
                          SPACE,              # the bounds on each dimension of x
                          n_initial_points=10,      # the starting point        ###acq_func="LCB",     # the acquisition function (optional)                          
                          n_calls=10,         # the number of evaluations of f including at x0                          
                          random_state=0)

CV MSE: 0.125:
CV MSE: 0.216:
CV MSE: 0.133:
CV MSE: 0.154:
CV MSE: 0.091:
CV MSE: 0.218:
CV MSE: 0.197:
CV MSE: 0.179:
CV MSE: 0.129:
CV MSE: 0.082:
CV MSE: 0.110:
CV MSE: 0.067:
CV MSE: 0.120:
CV MSE: 0.127:
CV MSE: 0.104:
CV MSE: 0.133:
CV MSE: 0.066:
CV MSE: 0.129:
CV MSE: 0.246:
CV MSE: 0.112:
Cross validation average MSE is 0.137:
CV MSE: 0.107:
CV MSE: 0.192:
CV MSE: 0.170:
CV MSE: 0.155:
CV MSE: 0.052:
CV MSE: 0.151:
CV MSE: 0.104:
CV MSE: 0.143:
CV MSE: 0.133:
CV MSE: 0.110:
CV MSE: 0.107:
CV MSE: 0.098:
CV MSE: 0.099:
CV MSE: 0.111:
CV MSE: 0.184:
CV MSE: 0.085:
CV MSE: 0.096:
CV MSE: 0.140:
CV MSE: 0.197:
CV MSE: 0.075:
Cross validation average MSE is 0.125:
CV MSE: 0.164:
CV MSE: 0.192:
CV MSE: 0.114:
CV MSE: 0.092:
CV MSE: 0.106:
CV MSE: 0.308:
CV MSE: 0.208:
CV MSE: 0.110:
CV MSE: 0.106:
CV MSE: 0.070:
CV MSE: 0.105:
CV MSE: 0.080:
CV MSE: 0.115:
CV MSE: 0.169:
CV MSE: 0.132:
CV MSE: 0.065:
CV MSE: 0.083:
CV MSE: 0.202:
CV MSE: 0.132:
CV MSE: 0.241:
Cross validation avera

In [ ]:
#Get Best Parameter Set
"Best score=%.4f" % results_gp.fun

print("""Best parameters:
- dense_0_neurons=%d
- dense_1_neurons=%d
- dropout_rate0=%.6f
- dropout_rate=%.6f""" % 
      (results_gp.x[0], results_gp.x[1],
                            results_gp.x[2], results_gp.x[3]))
# Model evaluation:
model_opt = get_model_reg(results_gp.x[0], results_gp.x[1], results_gp.x[2], results_gp.x[3], num_features, num_classes) 
score_test = model_opt.evaluate(X2_test, y2_test, verbose=0 ) 
print(' MSE on test dataset:', score_test[0])


Best parameters:
- dense_0_neurons=9
- dense_1_neurons=8
- dropout_rate0=0.214949
- dropout_rate=0.303446
 MSE on test dataset: 0.33775049448013306


The average of MSE on normalized training data is small (0.171) and MSE on normalized test data is reasonably small as well. 